In [2]:
from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

dn_input = np.load('/home/beto/Documents/projects/DoggoNet/DN-dataset.npz')
print(dn_input.files)

x_train = dn_input['X_train']
x_test = dn_input['X_test']
y_train = dn_input['y_train']
y_test = dn_input['y_test']

print(x_train.shape)

etiqueta=dn_input['y_train'][1200]
print(etiqueta[0])

foto=dn_input['X_train'][1200]
foto = np.multiply(foto, 255).astype(np.int8)
Image.fromarray(foto, 'RGB').show()

# convert class
num_classes = 4
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Define model
model = keras.Sequential(
    [
        keras.Input(shape=(240,320,3)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax")
    ]
)
model.summary()

batch_size = 200
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
print('ready')

['X_train', 'X_test', 'y_train', 'y_test']
(3400, 240, 320, 3)
2
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 238, 318, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 119, 159, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 117, 157, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 58, 78, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 289536)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 289536)            0         
_______________________________________________________

In [3]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
16/16 [==============================] - 104s 6s/step - loss: 3.4729 - accuracy: 0.3127 - val_loss: 1.0949 - val_accuracy: 0.6353
Epoch 2/15
16/16 [==============================] - 113s 7s/step - loss: 0.8118 - accuracy: 0.7114 - val_loss: 0.5847 - val_accuracy: 0.8647
Epoch 3/15
16/16 [==============================] - 113s 7s/step - loss: 0.4257 - accuracy: 0.8722 - val_loss: 0.3505 - val_accuracy: 0.8941
Epoch 4/15
16/16 [==============================] - 115s 7s/step - loss: 0.2842 - accuracy: 0.9072 - val_loss: 0.2845 - val_accuracy: 0.9294
Epoch 5/15
16/16 [==============================] - 118s 7s/step - loss: 0.1721 - accuracy: 0.9565 - val_loss: 0.2014 - val_accuracy: 0.9265
Epoch 6/15
16/16 [==============================] - 116s 7s/step - loss: 0.1057 - accuracy: 0.9742 - val_loss: 0.1998 - val_accuracy: 0.9353
Epoch 7/15
16/16 [==============================] - 117s 7s/step - loss: 0.0739 - accuracy: 0.9843 - val_loss: 0.1435 - val_accuracy: 0.9529
Epoch 8/15
16

In [5]:
model.save_weights('weights_CMCMFDrD_22feb')